In [1]:
import numpy as np
import pandas as pd
import quantecon as qe
from ast import literal_eval

In [2]:
def creating_samples(samples_number, element):
    gen=0
    uniform_number = len(element)
    uniform_samples = (-qe.simplex_grid(len(element), 1)+1)/(len(element)-1)
    if samples_number > uniform_number:
        random_samples = np.random.rand((samples_number-uniform_number), len(element))
        for i in range(samples_number-uniform_number):
            random_samples[i] = np.around(random_samples[i]/sum(random_samples[i]), decimals = 3)
        sample = np.vstack((uniform_samples, random_samples))
    else:
        sample = uniform_samples[:samples_number]
    samples = []
    for i in range(samples_number):
        samples.append(list(sample[i]))

    generation = []
    for i in range (samples_number):
            generation.append(gen)

    ID = np.arange(1, (samples_number+1))

    data = {'ID' : ID, 'Elements': [element], 'Generation': generation, 'Position': samples}
    df = pd.DataFrame(data=data, index = np.arange(samples_number))
    df.to_csv("Result/Initial Population.txt", sep='\t', index=False, mode='w')
    return df

In [3]:
samples_number = 6
element = ['Pt', 'Pd', 'Au']
creating_samples(samples_number, element)

,ID,Elements,Generation,Position
0,1,"[Pt, Pd, Au]",0,"[0.5, 0.5, 0.0]"
1,2,"[Pt, Pd, Au]",0,"[0.5, 0.0, 0.5]"
2,3,"[Pt, Pd, Au]",0,"[0.0, 0.5, 0.5]"
3,4,"[Pt, Pd, Au]",0,"[0.065, 0.476, 0.458]"
4,5,"[Pt, Pd, Au]",0,"[0.341, 0.3, 0.359]"
5,6,"[Pt, Pd, Au]",0,"[0.039, 0.517, 0.444]"


In [4]:
#Making Class
class pso:
    def __init__ (self, data, step, target = np.array([0.333, 0.333, 0.333])):
        self.df = data
        self.df['Elements'] = self.df["Elements"].apply(lambda x: literal_eval(x))
        self.position = []
        for i in range(len(self.df)):
            self.position.append(self.df["Position"].apply(lambda x: literal_eval(x))[i])
        self.position = np.array(self.position)
        self.df = self.df.drop(columns=['Position'])
        self.df = pd.concat([self.df, pd.DataFrame(([[i] for i in self.position]), columns = ['Position'])], axis = 1)
        self.velocity = np.around((-(self.position - target)/step), decimals = 3)
        self.df = pd.concat([self.df, pd.DataFrame(([[i] for i in self.velocity]), columns = ['Velocity'])], axis = 1)
        

        
    
    def move(self, epoch=1):
        self.upd_df = self.df
        for i in range(epoch):
            self.gen = self.upd_df['Generation'].max()
            self.now_df = self.upd_df.loc[self.upd_df['Generation']==self.gen]
            self.now_df = self.now_df.drop(columns = ['Generation'])
            self.now_df = self.now_df.assign(Generation = (self.gen + 1))
            self.now_position = []
            for i in self.now_df['Position']:
                self.now_position.append((list(i)))
            self.now_position = np.array(self.now_position)
            self.now_position = np.around((self.now_position + self.velocity), decimals = 3)
            self.now_df = self.now_df.drop(columns = ['Position'])
            self.now_df = pd.concat([self.now_df, pd.DataFrame(([[i] for i in self.now_position]), columns = ['Position'])], axis = 1)
            columns = ['ID', 'Generation', 'Elements', 'Position', 'Velocity']
            self.now_df = self.now_df[columns]
            self.upd_df = pd.concat([self.upd_df, self.now_df])
        return self.upd_df
    
    

In [5]:
step = 10
target = np.array([0.4, 0.3, 0.3])
population = pso(pd.read_csv('Result/Initial Population.txt', sep='\t'), step, target)

In [6]:
population.df

,ID,Elements,Generation,Position,Velocity
0,1,"[Pt, Pd, Au]",0,"[0.5, 0.5, 0.0]","[-0.01, -0.02, 0.03]"
1,2,"[Pt, Pd, Au]",0,"[0.5, 0.0, 0.5]","[-0.01, 0.03, -0.02]"
2,3,"[Pt, Pd, Au]",0,"[0.0, 0.5, 0.5]","[0.04, -0.02, -0.02]"
3,4,"[Pt, Pd, Au]",0,"[0.065, 0.476, 0.458]","[0.034, -0.018, -0.016]"
4,5,"[Pt, Pd, Au]",0,"[0.341, 0.3, 0.359]","[0.006, -0.0, -0.006]"
5,6,"[Pt, Pd, Au]",0,"[0.039, 0.517, 0.444]","[0.036, -0.022, -0.014]"


In [7]:
population.move(4)

,ID,Elements,Generation,Position,Velocity
0,1,"[Pt, Pd, Au]",0,"[0.5, 0.5, 0.0]","[-0.01, -0.02, 0.03]"
1,2,"[Pt, Pd, Au]",0,"[0.5, 0.0, 0.5]","[-0.01, 0.03, -0.02]"
2,3,"[Pt, Pd, Au]",0,"[0.0, 0.5, 0.5]","[0.04, -0.02, -0.02]"
3,4,"[Pt, Pd, Au]",0,"[0.065, 0.476, 0.458]","[0.034, -0.018, -0.016]"
4,5,"[Pt, Pd, Au]",0,"[0.341, 0.3, 0.359]","[0.006, -0.0, -0.006]"
5,6,"[Pt, Pd, Au]",0,"[0.039, 0.517, 0.444]","[0.036, -0.022, -0.014]"
0,1,"[Pt, Pd, Au]",1,"[0.49, 0.48, 0.03]","[-0.01, -0.02, 0.03]"
1,2,"[Pt, Pd, Au]",1,"[0.49, 0.03, 0.48]","[-0.01, 0.03, -0.02]"
2,3,"[Pt, Pd, Au]",1,"[0.04, 0.48, 0.48]","[0.04, -0.02, -0.02]"
3,4,"[Pt, Pd, Au]",1,"[0.099, 0.458, 0.442]","[0.034, -0.018, -0.016]"


In [ ]:
if self.df['Generation'].max() == 0:
    self.velocity = np.around((-(self.position - target)/step), decimals = 3)
    self.df = pd.concat([self.df, pd.DataFrame(([[i] for i in self.velocity]), columns = ['Velocity'])], axis = 1)
else:
    self.velocity = []
    for i in range(len(self.df)):
        self.velocity.append(self.df["Velocity"].apply(lambda x: literal_eval(x))[i])
    self.velocity = np.array(self.velocity)